In [1]:
import os
import time

import pandas as pd
import requests
import yfinance as yf
from datetime import datetime, timedelta, timezone
from pygooglenews import GoogleNews

from sec_edgar_downloader import Downloader
from pytrends.request import TrendReq

# Sentiment analysis
import finnhub
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# --- Directory constants ---
DATA_DIR = "../data/"
ROMONITOR_DIR = os.path.join(DATA_DIR, "romonitor_data")
SEC_DIR = os.path.join(DATA_DIR, "sec-edgar-filings")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(ROMONITOR_DIR, exist_ok=True)
os.makedirs(SEC_DIR, exist_ok=True)

# 1. Market Context Data
def update_market_context():
    tickers = ['^IXIC', '^VIX', '^TNX', 'XLC']
    # Use 1 year for context
    end = datetime.today()
    start = "2021-03-10"
    context = yf.download(tickers, start=start, end=end)['Close'].reset_index()
    context['date'] = context['Date'].dt.strftime('%Y-%m-%d')
    context = context.drop(columns=['Date'])
    context.to_csv(os.path.join(DATA_DIR, 'market_context_data.csv'), index=False)
    print("Updated market context data.")

# 2. Technical/Price Data
def update_technical():
    tickers = ["RBLX", "^GSPC", "^IXIC"]
    start_date = "2021-03-10"
    data = yf.download(tickers, start=start_date, group_by='ticker', auto_adjust=True)
    rblx = data['RBLX'].copy()
    # Add pandas_ta indicators if you want
    rblx.to_csv(os.path.join(DATA_DIR, 'RBLX_with_technicals.csv'))
    data['^GSPC'].to_csv(os.path.join(DATA_DIR, 'SP500.csv'))
    data['^IXIC'].to_csv(os.path.join(DATA_DIR, 'Nasdaq.csv'))

    print("Updated technical/price data.")

# 3. Sentiment/News Data (Finnhub)
def update_sentiment():
    api_key = os.environ.get('FINNHUB_KEY')
    client = finnhub.Client(api_key=api_key)

    DATA_FILE = '../data/clean_news_sentiment.csv'
    TARGET_FROM_DATE = datetime(2025, 6, 26)
    INITIAL_TO_DATE = datetime.today()  # Start from most recent

    # Helper to get the last date in the CSV (as datetime), or return INITIAL_TO_DATE if file doesn't exist
    def get_last_date():
        if os.path.exists(DATA_FILE):
            df = pd.read_csv(DATA_FILE)
            if not df.empty:
                min_date_str = df['date'].min()
                return datetime.strptime(min_date_str, '%Y-%m-%d')
        return INITIAL_TO_DATE

    while True:
        last_date = get_last_date()
        if last_date <= TARGET_FROM_DATE:
            print("All news up to target date fetched.")
            break

        from_date = max(TARGET_FROM_DATE, last_date - timedelta(days=30))  # Fetch up to 30 days at a time
        to_date = last_date

        print(f"Fetching news from {from_date.strftime('%Y-%m-%d')} to {to_date.strftime('%Y-%m-%d')}")

        news = client.company_news(
            "RBLX",
            _from=from_date.strftime("%Y-%m-%d"),
            to=to_date.strftime("%Y-%m-%d")
        )

        if not news:
            print("No more news to fetch.")
            break

        sia = SentimentIntensityAnalyzer()
        results = []
        for article in news:
            text = (article.get("headline", "") + "\n" + article.get("summary", "")).strip()
            vs = sia.polarity_scores(text)
            results.append({
                "date": datetime.fromtimestamp(article["datetime"]).strftime("%Y-%m-%d"),
                "headline": article["headline"],
                **vs
            })
        new_df = pd.DataFrame(results)

        # If file exists, append, else just save new
        if os.path.exists(DATA_FILE):
            old_df = pd.read_csv(DATA_FILE)
            combined_df = pd.concat([old_df, new_df], ignore_index=True)
            combined_df.drop_duplicates(subset=["date", "headline"], inplace=True)
        else:
            combined_df = new_df

        combined_df.to_csv(DATA_FILE, index=False)
        print(f"Appended news from {from_date.strftime('%Y-%m-%d')} to {to_date.strftime('%Y-%m-%d')}.")

        # To avoid too many requests per minute; adjust as needed to respect API rate limits
        import time
        time.sleep(60)

    print("Finished fetching all available news.")


# 4. User/Engagement Data (RoMonitor API)
def update_romonitor():
    # Always use UTC for timestamps in this API
    today = datetime.now(timezone.utc).strftime('%Y-%m-%dT23:59:59.999Z')

    endpoints = {
        "ccu": f"https://romonitorstats.com/api/v1/charts/get?name=platform-ccus&timeslice=day&start=2020-01-05T00:00:00.000Z&ends={today}",
        "registrations": f"https://romonitorstats.com/api/v1/charts/get?name=platform-registrations&timeslice=day&start=2020-03-15T00:00:00.000Z&ends={today}",
        "session_length": f"https://romonitorstats.com/api/v1/charts/get?name=platform-session-length&timeslice=day&start=2023-04-01T00:00:00.000Z&ends={today}"
    }
    output_dir = os.path.join("../data/", "romonitor_data")
    os.makedirs(output_dir, exist_ok=True)
    for key, url in endpoints.items():
        df = pd.DataFrame(requests.get(url).json())
        df.to_csv(os.path.join(output_dir, f"{key}.csv"), index=False)
        print(f"Updated {key} data from RoMonitor.")

# 5. Update and retrieve SEC data
def update_sec_data():
    dl = Downloader("RBLX_SEC", "saysaygo@gmail.com", "../data")
    CIK = "RBLX"

    dl.get("10-K", CIK, after="2021-01-01")
    dl.get("10-Q", CIK, after="2021-01-01")
    dl.get("8-K", CIK, after="2021-01-01")

    print("Updated SEC data")

# 6. Update Quarterly Financials
def update_quarterly_financials():
    rblx = yf.Ticker("RBLX")

    rblx.quarterly_financials.to_csv(DATA_DIR + "RBLX_quarterly_income_statement.csv")
    rblx.quarterly_balance_sheet.to_csv(DATA_DIR + "RBLX_quarterly_balance_sheet.csv")
    rblx.quarterly_cashflow.to_csv(DATA_DIR + "RBLX_quarterly_cashflow.csv")

    print("Updated Quarterly Financials")
    
def update_trends_data_daily_merged(max_retries=5, wait_seconds=60):
    """
    Downloads daily Google Trends data for 'Roblox' (web & news search)
    from 2021-03-10 to today, merges by date, and saves as a single CSV
    in YYYY-MM-DD format (e.g., 2021-03-31).
    """
    pytrends = TrendReq(hl='en-US', tz=360)
    kw_list = ["Roblox"]
    start_date = datetime(2021, 3, 10)
    end_date = datetime.today()
    window = timedelta(days=89)  # 90 days

    def get_trends(gprop, colname):
        all_data = []
        curr_start = start_date
        while curr_start < end_date:
            curr_end = min(curr_start + window, end_date)
            timeframe = f"{curr_start.strftime('%Y-%m-%d')} {curr_end.strftime('%Y-%m-%d')}"
            for attempt in range(max_retries):
                try:
                    pytrends.build_payload(kw_list, timeframe=timeframe, geo='', gprop=gprop)
                    df = pytrends.interest_over_time()
                    if df.empty:
                        print(f"No data for {colname} window {timeframe}. Skipping.")
                        break
                    if 'isPartial' in df.columns:
                        df = df.drop(columns=['isPartial'])
                    df = df.reset_index()
                    # Format as YYYY-MM-DD
                    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
                    df = df.rename(columns={'Roblox': colname})
                    df = df[['date', colname]]
                    all_data.append(df)
                    print(f"Fetched daily {colname} data for {timeframe}")
                    break
                except Exception as e:
                    print(f"Error on {colname} {timeframe}: {e}")
                    if attempt < max_retries - 1:
                        print(f"Retrying in {wait_seconds} seconds...")
                        time.sleep(wait_seconds)
                    else:
                        print(f"Giving up on {colname} window.")
            curr_start = curr_end + timedelta(days=1)
        if all_data:
            combined = pd.concat(all_data).drop_duplicates('date').sort_values('date')
            return combined
        else:
            print(f"No {colname} data retrieved.")
            return pd.DataFrame(columns=['date', colname])

    web_df = get_trends('', 'roblox_trend_web')
    news_df = get_trends('news', 'roblox_trend_news')

    merged = pd.merge(web_df, news_df, on='date', how='outer').sort_values('date').reset_index(drop=True)

    out_path = os.path.join(DATA_DIR, 'roblox_trends_merged_daily.csv')
    merged.to_csv(out_path, index=False)
    print(f"Saved merged daily Google Trends data for Roblox at {out_path}")

In [3]:
update_market_context()
update_technical()
update_sentiment()
update_romonitor()
#update_sec_data()
update_trends_data_daily_merged()
print("All data updated!")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  4 of 4 completed


Updated market context data.


[*********************100%***********************]  3 of 3 completed


Updated technical/price data.
Updated ccu data from RoMonitor.
Updated registrations data from RoMonitor.
Updated session_length data from RoMonitor.
Fetched daily roblox_trend_web data for 2021-03-10 2021-06-07
Fetched daily roblox_trend_web data for 2021-06-08 2021-09-05
Fetched daily roblox_trend_web data for 2021-09-06 2021-12-04
Fetched daily roblox_trend_web data for 2021-12-05 2022-03-04
Fetched daily roblox_trend_web data for 2022-03-05 2022-06-02
Fetched daily roblox_trend_web data for 2022-06-03 2022-08-31
Fetched daily roblox_trend_web data for 2022-09-01 2022-11-29
Fetched daily roblox_trend_web data for 2022-11-30 2023-02-27
Fetched daily roblox_trend_web data for 2023-02-28 2023-05-28
Fetched daily roblox_trend_web data for 2023-05-29 2023-08-26
Fetched daily roblox_trend_web data for 2023-08-27 2023-11-24
Fetched daily roblox_trend_web data for 2023-11-25 2024-02-22
Fetched daily roblox_trend_web data for 2024-02-23 2024-05-22
Fetched daily roblox_trend_web data for 2024

### Just In-case update_sentiment doesn't work :)

In [4]:
import datetime, time, random, urllib.parse, requests, feedparser, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from email.utils import parsedate_to_datetime

BASE = "https://news.google.com/rss/search"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_1) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/125.0.0.0 Safari/537.36"
    ),
    "Accept": "application/rss+xml, text/xml;q=0.9",
}

def month_slices(start, end):
    d = datetime.date(start.year, start.month, 1)
    while d <= end:
        nxt = (d.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
        yield d, min(nxt - datetime.timedelta(days=1), end)
        d = nxt

def build_url(keywords, since, until):
    q = f'{keywords} after:{since} before:{(until + datetime.timedelta(days=1))}'
    params = {"q": q, "hl": "en-US", "gl": "US", "ceid": "US:en"}
    return f"{BASE}?{urllib.parse.urlencode(params, quote_via=urllib.parse.quote_plus)}"

def session(max_retries=5):
    s = requests.Session()
    rs = Retry(total=max_retries, backoff_factor=1,
               status_forcelist=[429, 500, 502, 503, 504])
    s.mount("https://", HTTPAdapter(max_retries=rs))
    s.headers.update(HEADERS)
    return s

def fetch_month(url, sess):
    for _ in range(2):                          # up to 1 retry on 503
        r = sess.get(url, timeout=15)
        if r.status_code == 200:
            return feedparser.parse(r.text).entries
        if r.status_code == 503:
            time.sleep(60)                      # hard back-off
    raise RuntimeError(f"Still 503 → {url}")

def scrape(keywords, start, end, outfile):
    sia, rows, sess = SentimentIntensityAnalyzer(), [], session()
    for since, until in month_slices(start, end):
        url = build_url(keywords, since, until)
        for e in fetch_month(url, sess):
            score = sia.polarity_scores(e.title)
            rows.append({
                "date": parsedate_to_datetime(e.published).strftime("%Y-%m-%d"),
                "headline": e.title, 
                **score
            })
        time.sleep(random.uniform(6, 12))       # polite spacing

    new_df = pd.DataFrame(rows)

    # Check if file exists
    if os.path.exists(outfile):
        old_df = pd.read_csv(outfile)
        # Combine new data on top of old data
        combined_df = pd.concat([new_df, old_df], ignore_index=True)
        # Drop duplicates (by date+headline, keep first occurrence)
        combined_df.drop_duplicates(subset=["date", "headline"], keep="first", inplace=True)
    else:
        combined_df = new_df

    # Sort so latest date is on top
    combined_df["date"] = pd.to_datetime(combined_df["date"])
    combined_df.sort_values(by="date", ascending=False, inplace=True)
    combined_df.to_csv(outfile, index=False)
    print(f"Saved {len(combined_df)} rows → {outfile}")

# -------- run --------
ipo  = datetime.date(2025, 6, 23)
today = datetime.date.today()
scrape("RBLX stock", ipo, today, "../data/clean_news_sentiment.csv")

Saved 2366 rows → ../data/clean_news_sentiment.csv


### Data Cleaning Part 1

In [5]:
import pandas as pd
import ast
import os

from bs4.diagnose import lxml_trace

DATA_DIR = "../data/romonitor_data/"
OUTPUT_DIR = "../data/romonitor_data_clean/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def extract_timeseries_from_data(file, value_colname):
    df = pd.read_csv(DATA_DIR + file)
    # Only keep rows with a real 'name' (not metadata)
    df = df[df['name'].notna()]
    # Parse the dict in 'data'
    data_dict = ast.literal_eval(df.iloc[0]['data'])
    # Convert to DataFrame
    ts_df = pd.DataFrame(list(data_dict.items()), columns=['date', value_colname])
    ts_df['date'] = pd.to_datetime(ts_df['date']).dt.strftime('%Y-%m-%d')
    ts_df = ts_df.sort_values('date').reset_index(drop=True)
    ts_df.to_csv(OUTPUT_DIR + value_colname + '.csv', index=False)
    print(f"Saved: {OUTPUT_DIR}{value_colname}.csv")
    return ts_df

def extract_and_sum_popular_games():
    df = pd.read_csv(DATA_DIR + 'popular_games.csv')
    df = df[df['name'].notna()]
    game_dfs = []
    for _, row in df.iterrows():
        data_raw = row['data']
        try:
            data_dict = ast.literal_eval(data_raw)
        except:
            continue  
        if isinstance(data_dict, dict):
            game_df = pd.DataFrame(list(data_dict.items()), columns=['date', 'value'])
            game_df['date'] = pd.to_datetime(game_df['date'])
            game_dfs.append(game_df)
    if not game_dfs:
        print("No games with usable data found!")
        return None
    all_games = pd.concat(game_dfs)
    sum_games = all_games.groupby(all_games['date'].dt.strftime('%Y-%m-%d'))['value'].sum().reset_index()
    sum_games = sum_games.rename(columns={'value': 'popular_games_total'})
    sum_games.to_csv(OUTPUT_DIR + 'popular_games_total.csv', index=False)
    print(f"Saved: {OUTPUT_DIR}popular_games_total.csv")
    return sum_games

# Clean and save time series
extract_timeseries_from_data('ccu.csv', 'ccu')
extract_timeseries_from_data('registrations.csv', 'registrations')
extract_timeseries_from_data('session_length.csv', 'session_length')
extract_and_sum_popular_games()

Saved: ../data/romonitor_data_clean/ccu.csv
Saved: ../data/romonitor_data_clean/registrations.csv
Saved: ../data/romonitor_data_clean/session_length.csv
Saved: ../data/romonitor_data_clean/popular_games_total.csv


,date,popular_games_total
0,2025-05-06,79758549.0
1,2025-05-07,79997871.0
2,2025-05-08,81165810.0
3,2025-05-09,89288600.0
4,2025-05-10,116785967.0
5,2025-05-11,114392357.0
6,2025-05-12,79589010.0
7,2025-05-13,75347467.0
8,2025-05-14,75645524.0
9,2025-05-15,75085390.0


In [6]:
def clean_quarterly_fundamental(filepath, prefix=None):
    # Load and transpose so dates are rows, metrics are columns
    df = pd.read_csv(filepath, index_col=0).transpose()

    df = df.reset_index().rename(columns={'index': 'date'})
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    if prefix:
        newcols = ['date'] + [f"{prefix}{col}" for col in df.columns if col != 'date']
        df.columns = newcols
    df = df.sort_values('date').reset_index(drop=True)
    return df

# EXAMPLES:
# Cleaned income statement
qis = clean_quarterly_fundamental("../data/RBLX_quarterly_income_statement.csv", prefix="qis_")
qis.to_csv("../data/cleaned_quarterly_income_statement.csv", index=False)

# Cleaned balance sheet
qbs = clean_quarterly_fundamental("../data/RBLX_quarterly_balance_sheet.csv", prefix="qbs_")
qbs.to_csv("../data/cleaned_quarterly_balance_sheet.csv", index=False)

# Cleaned cash flow
qcf = clean_quarterly_fundamental("../data/RBLX_quarterly_cashflow.csv", prefix="qcf_")
qcf.to_csv("../data/cleaned_quarterly_cashflow.csv", index=False)

In [9]:
# Parsing and extracting SEC file data

import os
import re
import glob
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import textstat

nltk.download('punkt_tab')
nltk.download('stopwords')

# --- Load Loughran-McDonald financial sentiment dictionary ---
def load_lm_dicts(lm_dict_path):
    lm_df = pd.read_csv(lm_dict_path)
    lm_negative = set(lm_df[lm_df['Negative'] > 0]['Word'].str.lower())
    lm_positive = set(lm_df[lm_df['Positive'] > 0]['Word'].str.lower())
    return lm_negative, lm_positive

def extract_filing_date_from_txt(raw_txt):
    # Look for the "FILED AS OF DATE:" line and extract date
    match = re.search(r'FILED AS OF DATE:\s+(\d{8})', raw_txt)
    if match:
        # Format as YYYY-MM-DD
        date_str = match.group(1)
        return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
    return ""  # fallback if not found

def clean_text(text):
    # Remove HTML tags, non-letters, numbers, extra whitespace
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return text.strip()

def compute_lm_sentiment(tokens, neg_words, pos_words):
    tokens = [t.lower() for t in tokens if len(t) > 1]
    n_pos = sum(1 for t in tokens if t in pos_words)
    n_neg = sum(1 for t in tokens if t in neg_words)
    if n_pos + n_neg == 0:
        return 0
    return (n_pos - n_neg) / (n_pos + n_neg)

def extract_date_and_type(filename):
    # Try to extract date (YYYY-MM-DD or YYYYMMDD) and form type from filename or path
    date_match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)
    if not date_match:
        date_match = re.search(r'(\d{8})', filename)
        if date_match:
            # Convert YYYYMMDD to YYYY-MM-DD
            date_str = date_match.group(1)
            date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        else:
            date = ""
    else:
        date = date_match.group(1)
    # Form type: look for 10-K, 10-Q, 8-K in path or filename
    type_match = re.search(r'(10[-\s]?K|10[-\s]?Q|8[-\s]?K)', filename, re.IGNORECASE)
    form_type = type_match.group(1).upper().replace(' ', '').replace('-', '') if type_match else ""
    return date, form_type

def process_filing(filepath, neg_words, pos_words):
    basename = os.path.basename(filepath)
    with open(filepath, encoding='utf-8', errors='ignore') as f:
        raw = f.read()
    filing_date = extract_filing_date_from_txt(raw)
    form_type = extract_date_and_type(filepath)[1]
    text = clean_text(raw)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stopwords.words('english')]
    try:
        fog = textstat.gunning_fog(text)
    except Exception:
        fog = None
    return {
        'date': filing_date,
        'form_type': form_type,
        'filename': basename,
        'fog_index': fog,
    }

def process_all_filings(folder, lm_dict_path, out_csv):
    neg_words, pos_words = load_lm_dicts(lm_dict_path)
    results = []
    for root, dirs, files in os.walk(folder):
        for fname in files:
            if fname.endswith('.txt'):
                filepath = os.path.join(root, fname)
                summary = process_filing(filepath, neg_words, pos_words)
                results.append(summary)
    df = pd.DataFrame(results)
    df = df.sort_values('date').reset_index(drop=True)
    df.to_csv(out_csv, index=False)
    print(f"Saved SEC filings features to {out_csv}")
    return df

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\saysa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saysa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
SEC_FOLDER = "../data/sec-edgar-filings/"
LM_DICT_PATH = "../data/Loughran-McDonald_MasterDictionary_1993-2024.csv"
OUT_CSV = "../data/sec_filings_features.csv"
process_all_filings(SEC_FOLDER, LM_DICT_PATH, OUT_CSV)

KeyboardInterrupt: 

In [7]:
# Merge All the data together

import pandas as pd

def standardize_date_column(df):
    # Rename any column with 'date' (case-insensitive) to 'date'
    for col in df.columns:
        if col.lower() == 'date':
            df = df.rename(columns={col: 'date'})
            break
    return df

def move_to_monday(date):
    if date.weekday() == 5:  # Saturday
        return date + pd.Timedelta(days=2)
    elif date.weekday() == 6:  # Sunday
        return date + pd.Timedelta(days=1)
    else:
        return date

# --- Load Data ---
stock = pd.read_csv("../data/RBLX_with_technicals.csv")
sp500 = pd.read_csv("../data/SP500.csv")
nasdaq = pd.read_csv("../data/Nasdaq.csv")
popular_games_total = pd.read_csv("../data/romonitor_data_clean/popular_games_total.csv")
registrations = pd.read_csv("../data/romonitor_data_clean/registrations.csv")
ccu = pd.read_csv("../data/romonitor_data_clean/ccu.csv")
session_length = pd.read_csv("../data/romonitor_data_clean/session_length.csv")
sentiment = pd.read_csv("../data/clean_news_sentiment.csv")
market = pd.read_csv("../data/market_context_data.csv")
trends = pd.read_csv("../data/roblox_trends_merged_daily.csv")
sec = pd.read_csv("../data/sec_filings_features.csv")
qbs = pd.read_csv("../data/cleaned_quarterly_balance_sheet.csv")
qcf = pd.read_csv("../data/cleaned_quarterly_cashflow.csv")
qis = pd.read_csv("../data/cleaned_quarterly_income_statement.csv")

# --- Standardize Columns and Dates ---
sp500 = standardize_date_column(sp500)
nasdaq = standardize_date_column(nasdaq)
sp500 = sp500.rename(columns={col: f"SP500_{col}" for col in sp500.columns if col != "date"})
nasdaq = nasdaq.rename(columns={col: f"Nasdaq_{col}" for col in nasdaq.columns if col != "date"})

# --- Aggregating sentiment scores ---
sentiment['date'] = pd.to_datetime(sentiment['date'], errors='coerce')
sentiment['date'] = sentiment['date'].apply(move_to_monday)
sentiment = sentiment.groupby('date', as_index=False)[['neg', 'neu', 'pos', 'compound']].mean()

trends['date'] = pd.to_datetime(trends['date'], errors='coerce')
trends['date'] = trends['date'].apply(move_to_monday)
trends = trends.groupby('date', as_index=False)[['roblox_trend_web', 'roblox_trend_news']].mean()


dfs = [
    stock, sp500, nasdaq, popular_games_total, registrations,
    ccu, session_length, sentiment, trends, market, sec
]
dfs = [standardize_date_column(df) for df in dfs]

# Add quarterly dfs to standardize and convert dates too (but don't merge yet)
quarterly_dfs = [qbs, qcf, qis]
quarterly_dfs = [standardize_date_column(df) for df in quarterly_dfs]

# Ensure all 'date' columns are datetime type
for df in dfs + quarterly_dfs:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

# --- Merge All Daily DataFrames ---
master = dfs[0]
for df in dfs[1:]:
    master = master.merge(df, on='date', how='outer')

# --- Forward-fill Quarterly Data to Daily Rows ---
for quarterly_df in quarterly_dfs:
    quarterly_df = quarterly_df.sort_values('date')
    master = pd.merge_asof(master.sort_values('date'), quarterly_df, on='date', direction='backward')

# --- Filter to IPO date or later ---
ipo_date = pd.to_datetime("2021-03-10")
master = master[master["date"] >= ipo_date]

# --- Final Touches ---
master = master.sort_values('date').reset_index(drop=True)
master.to_csv("../data/master_dataset.csv", index=False)
print("Master dataset saved as master_dataset.csv (with quarterly features forward-filled)")

Master dataset saved as master_dataset.csv (with quarterly features forward-filled)


### Data Cleaning Part 2

In [8]:
import pandas as pd
import numpy as np

master = pd.read_csv("../data/master_dataset.csv")

In [9]:
# 2. Drop rows where the market is closed (no stock price)
master = master[master['Close'].notna()].copy()

master

,date,Open,High,Low,Close,Volume,SP500_Open,SP500_High,SP500_Low,SP500_Close,...,qis_Other Operating Expenses,qis_Research And Development,qis_Selling General And Administration,qis_Selling And Marketing Expense,qis_General And Administrative Expense,qis_Other Gand A,qis_Gross Profit,qis_Cost Of Revenue,qis_Total Revenue,qis_Operating Revenue
0,2021-03-10,64.500000,74.830002,60.500000,69.500000,97069300.0,3891.989990,3917.350098,3885.729980,3898.810059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-11,74.930000,77.779999,70.129997,73.900002,59629300.0,3915.540039,3960.270020,3915.540039,3939.340088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-12,72.470001,72.959999,69.110001,69.699997,19714700.0,3924.520020,3944.989990,3915.209961,3943.340088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-03-15,70.019997,74.059998,66.250000,72.150002,19549800.0,3942.959961,3970.080078,3923.540039,3968.939941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-03-16,73.730003,78.000000,73.180000,77.000000,30274400.0,3973.590088,3981.040039,3953.439941,3962.709961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,2025-07-10,106.900002,107.370003,102.730003,105.029999,5636700.0,6266.799805,6290.220215,6251.439941,6280.459961,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09
1592,2025-07-11,104.949997,108.099998,104.809998,105.690002,4544500.0,6255.680176,6269.439941,6237.600098,6259.750000,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09
1595,2025-07-14,106.349998,111.959999,106.349998,111.830002,8032300.0,6255.149902,6273.310059,6239.220215,6268.560059,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09
1596,2025-07-15,110.610001,113.070000,110.209999,112.480003,8105900.0,6295.290039,6302.040039,6241.680176,6243.759766,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09


In [10]:
# 3. Forward-fill quarterly features (qbs_, qcf_, qis_)
quarterly_cols = [col for col in master.columns if any(q in col.lower() for q in ['qbs_', 'qcf_', 'qis_'])]
if quarterly_cols:
    master[quarterly_cols] = master[quarterly_cols].ffill()

master

,date,Open,High,Low,Close,Volume,SP500_Open,SP500_High,SP500_Low,SP500_Close,...,qis_Other Operating Expenses,qis_Research And Development,qis_Selling General And Administration,qis_Selling And Marketing Expense,qis_General And Administrative Expense,qis_Other Gand A,qis_Gross Profit,qis_Cost Of Revenue,qis_Total Revenue,qis_Operating Revenue
0,2021-03-10,64.500000,74.830002,60.500000,69.500000,97069300.0,3891.989990,3917.350098,3885.729980,3898.810059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-11,74.930000,77.779999,70.129997,73.900002,59629300.0,3915.540039,3960.270020,3915.540039,3939.340088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-12,72.470001,72.959999,69.110001,69.699997,19714700.0,3924.520020,3944.989990,3915.209961,3943.340088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-03-15,70.019997,74.059998,66.250000,72.150002,19549800.0,3942.959961,3970.080078,3923.540039,3968.939941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-03-16,73.730003,78.000000,73.180000,77.000000,30274400.0,3973.590088,3981.040039,3953.439941,3962.709961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,2025-07-10,106.900002,107.370003,102.730003,105.029999,5636700.0,6266.799805,6290.220215,6251.439941,6280.459961,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09
1592,2025-07-11,104.949997,108.099998,104.809998,105.690002,4544500.0,6255.680176,6269.439941,6237.600098,6259.750000,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09
1595,2025-07-14,106.349998,111.959999,106.349998,111.830002,8032300.0,6255.149902,6273.310059,6239.220215,6268.560059,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09
1596,2025-07-15,110.610001,113.070000,110.209999,112.480003,8105900.0,6295.290039,6302.040039,6241.680176,6243.759766,...,523691000.0,374600000.0,166900000.0,47768000.0,119132000.0,119132000.0,810482000.0,224725000.0,1.035207e+09,1.035207e+09


In [11]:
# 4. Forward-fill/interpolate daily user/engagement features
daily_fill_cols = [
    'registrations', 'ccu', 'session_length', 'popular_games_total'
]
daily_fill_cols = [col for col in daily_fill_cols if col in master.columns]
if daily_fill_cols:
    master[daily_fill_cols] = master[daily_fill_cols].interpolate(method='linear', limit_direction='both')
    master[daily_fill_cols] = master[daily_fill_cols].ffill().bfill()

In [12]:
master['date'] = pd.to_datetime(master['date'], errors='coerce')
# Drop duplicate dates
master = master.drop_duplicates(subset=['date'], keep='first')
master = master.sort_values('date').reset_index(drop=True)
master = master.drop(columns=['fog_index', 'filename', 'form_type'])

In [13]:
def clean_numeric(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace('<null>', '').replace('None', '').replace('', '')
    # Replace Unicode dashes with ASCII minus
    val = val.replace('\u2013', '-')  # EN DASH
    val = val.replace('\u2014', '-')  # EM DASH
    val = val.replace('\u2212', '-')  # MINUS SIGN
    val = val.replace(',', '')        # Remove commas (thousands separators)
    val = val.replace('(', '-')       # Handle accounting negatives
    val = val.replace(')', '')
    val = val.strip()
    if val == '':
        return np.nan
    try:
        return float(val)
    except Exception:
        return np.nan

# Replace all null-like values globally with np.nan
master = master.replace({'<null>': np.nan, 'nan': np.nan, 'None': np.nan, '': np.nan})

# Clean ALL columns except 'date'
for col in master.columns:
    if col.lower() == 'date':
        continue
    master[col] = master[col].apply(clean_numeric)

In [14]:
# 7. Save the cleaned dataset
master.to_csv("../data/master_dataset_cleaned.csv", index=False)
print("Cleaned master dataset saved as master_dataset_cleaned.csv")

Cleaned master dataset saved as master_dataset_cleaned.csv


In [15]:
# 8. Check quarterly feature filling worked (print first 40 non-NaN rows)
if quarterly_cols:
    quarterly_data = master[['date'] + quarterly_cols]
    print("\nFirst rows with quarterly features filled:")
    print(quarterly_data[quarterly_data[quarterly_cols[0]].notna()].head(40))


First rows with quarterly features filled:
         date  qbs_Ordinary Shares Number  qbs_Share Issued  qbs_Net Debt  \
15 2021-03-31                 568894000.0       568894000.0  1.600000e+09   
16 2021-04-01                 568894000.0       568894000.0  1.600000e+09   
17 2021-04-05                 568894000.0       568894000.0  1.600000e+09   
18 2021-04-06                 568894000.0       568894000.0  1.600000e+09   
19 2021-04-07                 568894000.0       568894000.0  1.600000e+09   
20 2021-04-08                 568894000.0       568894000.0  1.600000e+09   
21 2021-04-09                 568894000.0       568894000.0  1.600000e+09   
22 2021-04-12                 568894000.0       568894000.0  1.600000e+09   
23 2021-04-13                 568894000.0       568894000.0  1.600000e+09   
24 2021-04-14                 568894000.0       568894000.0  1.600000e+09   
25 2021-04-15                 568894000.0       568894000.0  1.600000e+09   
26 2021-04-16                 56

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# 1. Load & sort your data
df = pd.read_csv('../data/master_dataset_cleaned.csv', parse_dates=['date'])
df.sort_values('date', inplace=True)
df.set_index('date', inplace=True)

# 2. Specify the sentiment columns and bootstrap-missing with column means
sent_cols = ['neg', 'neu', 'pos', 'compound']
col_means = df[sent_cols].mean()
df_init = df[sent_cols].fillna(col_means)

# 3. Scale into [0,1] for the LSTM
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_init)

# 4. Build sequences (look_back days → predict next day)
def create_sequences(arr, look_back=7):
    X, y = [], []
    for i in range(look_back, len(arr)):
        X.append(arr[i-look_back:i])
        y.append(arr[i])
    return np.array(X), np.array(y)

look_back = 7
X, y = create_sequences(data_scaled, look_back)

# 5. Train/test split
dataset = TensorDataset(torch.tensor(X, dtype=torch.float32),
                        torch.tensor(y, dtype=torch.float32))
train_size = int(len(dataset) * 0.8)
train_ds, test_ds = random_split(dataset, [train_size, len(dataset)-train_size])
train_loader = DataLoader(train_ds, batch_size=16, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=16)

# 6. Define your LSTM imputer
class ImputerLSTM(nn.Module):
    def __init__(self, n_features, hidden_size=32):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden_size, batch_first=True)
        self.out  = nn.Linear(hidden_size, n_features)
    def forward(self, x):
        # x: [batch, seq_len, features]
        _, (h_n, _) = self.lstm(x)
        return self.out(h_n[-1])  # use last layer's final hidden

model = ImputerLSTM(n_features=len(sent_cols), hidden_size=32)

# 7. Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
best_val = float('inf')
patience = 5
wait = 0
best_state = None

# 8. Train with early stopping
for epoch in range(1, 51):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()

    # validation
    model.eval()
    with torch.no_grad():
        val_losses = [criterion(model(xb), yb).item()
                      for xb, yb in test_loader]
    val_loss = np.mean(val_losses)

    if val_loss < best_val:
        best_val = val_loss
        best_state = model.state_dict()
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"Stopping early at epoch {epoch}")
            break

# restore best
model.load_state_dict(best_state)
model.eval()

# 9. Impute missing rows
missing_dates = df[df[sent_cols].isnull().any(axis=1)].index
all_scaled = torch.tensor(data_scaled, dtype=torch.float32)

for dt in missing_dates:
    idx = df.index.get_loc(dt)
    if idx >= look_back:
        seq = all_scaled[idx-look_back:idx].unsqueeze(0)  # [1, look_back, features]
        with torch.no_grad():
            pred_scaled = model(seq).numpy()
        df.loc[dt, sent_cols] = scaler.inverse_transform(pred_scaled)[0]

# 10. Check results
print("Remaining NaNs per column:\n", df[sent_cols].isna().sum())
print("\nFirst few imputed rows:")
print(df.loc[missing_dates[:5], sent_cols])

df.to_csv('../data/master_dataset_cleaned.csv', index=True)



Stopping early at epoch 34
Remaining NaNs per column:
 neg         0
neu         0
pos         0
compound    0
dtype: int64

First few imputed rows:
                 neg       neu       pos  compound
date                                              
2021-03-23  0.045944  0.861251  0.087522  0.136717
2021-03-24  0.046739  0.857825  0.089266  0.127215
2021-03-26  0.064049  0.843266  0.085045  0.058122
2021-03-31  0.077096  0.833341  0.083796  0.009647
2021-04-01  0.067255  0.834247  0.091124  0.032901
